In [1]:
%load_ext autoreload
%autoreload 2   # Change to %autoreload when development phase is over

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Define working environment

**Import libraries**

In [2]:
## Import libraries needed for setting parameters of operating system 
import os
import sys
## Import library for temporary files creation 
import tempfile 
## Import library for time management 
import time

In [3]:
## Import Pandas library
import pandas as pd
## Import Numpy library
import numpy as np

In [4]:
## Import Matplotlib 
import matplotlib as mpl 
## agg backend is used to create plot as a .png file
mpl.use('agg')
## Import Matplotlib.pyplot for creating graphs
import matplotlib.pyplot as plt 

In [5]:
%matplotlib inline

** Add folder with SCR provided belong to this notebook**

In [6]:
# Add local module to the path
src = os.path.abspath('../SRC')
if src not in sys.path:
    sys.path.append(src)

** Environment variables when working on Linux Mint **

In [7]:
import environ_variables as envi

In [8]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

MDMSESSION = mate 	
MANDATORY_PATH = /usr/share/gconf/mate.mandatory.path 	
MATE_DESKTOP_SESSION_ID = this-is-deprecated 	
LESSOPEN = | /usr/bin/lesspipe %s 	
MDM_LANG = fr_BE.UTF-8 	
LOGNAME = tais 	
USER = tais 	
HOME = /home/tais 	
XDG_VTNR = 9 	
PATH = /usr/local/bin:/home/tais/BIN:/home/tais/bin:/home/tais/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/bin:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/script:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/lib 	
CLICOLOR = 1 	
DISPLAY = :0.0 	
SSH_AGENT_PID = 25466 	
LANG = fr_BE.UTF-8 	
TERM = xterm-color 	
SHELL = /bin/bash 	
GIS_LOCK = $$ 	
XAUTHORITY = /home/tais/.Xauthority 	
SESSION_MANAGER = local/tais-HP-Z620-Workstation:@/tmp/.ICE-unix/25383,unix/tais-HP-Z620-Workstation:/tmp/.ICE-unix/25383 	
SHLVL = 1 	
QT_LINUX_ACCESSIBILITY_ALWAYS_ON = 1 	
INSIDE_CAJA_PYTHON =  	
QT_ACCESSIBILITY = 1 	
LD

** GRASS GIS Python libraries **

In [9]:
## Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
## Import libraries needed to call GRASS using Python
import grass.script as gscript

** Functions defined by the user **

In [10]:
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb
from grass_database import check_location
from grass_database import check_mapset
from grass_database import working_mapset

In [11]:
## Import functions for processing time information
from processing_time import start_processing
from processing_time import print_processing_time

In [12]:
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name

In [13]:
# Import function that create color file for raster
from colorise_raster import create_color_rule

In [23]:
# Import function that clip multiple raster according to extention of a vector layer
from create_clumped_grid import create_clumped_grid

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# User inputs

Here after:
- Enter the path to the directory you want to use as "[GRASSDATA](https://grass.osgeo.org/programming7/loc_struct.png)". 
- Enter the name of the location in which you want to work and its projection information in [EPSG code](http://spatialreference.org/ref/epsg/) format. Please note that the GRASSDATA folder and locations will be automatically created if not existing yet. If the location name already exists, the projection information will not be used.  
- Enter the name you want for the mapsets which will be used later for Unsupervised Segmentation Parameter Optimization (USPO), Segmentation and Classification steps.

In [24]:
## Define a empty dictionnary for saving user inputs
user={}

In [30]:
## Enter the path to GRASSDATA folder
user["gisdb"] = "/media/tais/My_Book_1/MAUPP/Traitement/Population_modelling_dasymetry/GRASSDATA"
## Enter the name of the location (existing or for a new one)
user["location"] = "Dakar_32628"
## Enter the EPSG code for this location 
user["locationepsg"] = "32628"
## Enter the name of the permanent mapset
user["permanent_mapset"] = "PERMANENT"
## Enter the name of a working mapset
user["dasym_mapset"] = "DASYMETRY"

**Check for existance of GRASSDATA folder, location and mapsets**

Here after, the python script will check if GRASSDATA folder, locations and mapsets already exist. If not, they will be automatically created.

In [31]:
# Check if the GRASS GIS database exists and create it if not
check_gisdb(user["gisdb"])

GRASSDATA folder already exist


In [32]:
# Check if the location exists and create it if not, with the CRS defined by the epsg code 
check_location(user["gisdb"],user["location"],user["locationepsg"])

Location Dakar_32628 already exist


In [33]:
# Check if the mapset exists and create it if not
check_mapset(user["gisdb"],user["location"],user["permanent_mapset"])

'PERMANENT' mapset already exists in location 'Dakar_32628'


In [34]:
# Check if the mapset exists and create it if not
check_mapset(user["gisdb"],user["location"],user["dasym_mapset"])

'DASYMETRY' mapset already exists in location 'Dakar_32628'


**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Launch GRASS GIS working session on DASYMETRY mapset

In [35]:
# Change the current working GRASS GIS session mapset
working_mapset(user["gisdb"],user["location"],user["dasym_mapset"])

You are now working in mapset 'DASYMETRY'


### Set the name of main layers

In [70]:
# Set the name of the layers
lc = 'landcover'  # VHR Land cover map
lu = 'landuse' # VHR Land use map
mr_builtup = 'MR_builtup' # MR built-up map
grid = 'clumped_grid'
## Name of the column containing the population count
population_column="POPULATION"

### Set the resolution of the prediction grid

In [79]:
tile_size = 100

### Create temporary directories for output

In [112]:
import os

def create_tempdirs(list_of_directories):
    '''
    Function that create needed temporary folder. Those name have to be saved as other function will depend of the name of those folder.
    '''
    return_list = []
    tmp_grass_dir=gscript.tempdir()
    
    for directory in list_of_directories:    
        # Temporary directory for administrative units statistics
        outputdirectory=os.path.join(tmp_grass_dir,directory)
        if not os.path.exists(outputdirectory):
            os.makedirs(outputdirectory)
        return_list.append(outputdirectory)
    # Return paths
    return return_list

In [113]:
# Create temporary folder and get their paths
outputdirectory_admin, outputdirectory_grid = create_tempdirs(["test", "grid_level"])

# Create a clumped grid for dasymetry

This Grid will be the reference grid for re-allocation of population count.

In [42]:
# Create clumped grid
create_clumped_grid(tile_size=tile_size, align_raster=lc, output='clumped_grid')

## Create layers with grid boundary for both levels

In [91]:
def gridded_admin_boundaries(input_vector, id, pop_column, grid):
    '''
    Function convecting the vector to raster then raster to vector: boundaries will have a "staircase" appearence
    so that each tile of the gridded vector will be contained in only one administrative unit
    '''
    
    def check_no_missing_zones(vector_origin, vector_gridded, resolution):
        '''
        Function checking if the number of items (admin zones) in the original vector provided by the user is wall conserved after the rasterization.
        If the original vector contains small sized polygons (or very tight) and desired 'tile_size' is too large, some polygons could disappeared during the rasterization process
        '''
        origin_n=gscript.parse_command('v.db.univar', flags='g', map=vector_origin, column='cat')['n']
        gridded_n=gscript.parse_command('v.db.univar', flags='g', map=vector_gridded, column='cat')['n']
        if origin_n != gridded_n:
            gscript.run_command('g.remove', quiet=True, type='vector', name=vector_gridded, flags='fb')
            message=_(("A tile size of %s m seems too large and produce loss of some polygons when rasterizing them.\n") % resolution)
            message+=_(("Try to reduce the 'tile_size' parameter or edit the <%s> vector to merge smallest administrative units with their neighoring units") % vector_origin)
            gscript.fatal(message)
        
    current_mapset = gscript.gisenv()['MAPSET']
    gscript.run_command('g.region', raster=grid)
    resolution = int(float(gscript.parse_command('g.region', flags='pg')['nsres']))
    tmp_rast = random_layer_name(prefix='tmp_rast_%s'%input_vector.split("@")[0])
    gridded_vector = input_vector.split("@")[0]+'_'+str(resolution)+'m_gridded'+'@'+current_mapset
    gscript.run_command('v.to.rast', quiet=True, input=input_vector, type='area', 
                        output=tmp_rast, use='attr', 
                        attribute_column=id, overwrite=True)
    gscript.run_command('r.to.vect', quiet=True, input=tmp_rast, 
                        output=gridded_vector, type='area', column=id, 
                        flags='v',overwrite=True)
    tmp_name=random_layer_name()
    gscript.run_command('g.copy', quiet=True, vector='%s,%s'%(input_vector,tmp_name))
    gscript.run_command('v.db.join', quiet=True, map_=gridded_vector, column='cat', other_table=tmp_name, other_column=id, subset_columns=pop_column) #join the population count
    gscript.run_command('g.remove', quiet=True, flags='f', type='vector', name=tmp_name+'@'+current_mapset)
    check_no_missing_zones(input_vector, gridded_vector, resolution)    
    gscript.run_command('g.remove', quiet=True, flags='f',type='raster',name=tmp_rast)

In [92]:
# Create layer with boundaries corresponding to the grid 
gridded_admin_boundaries("admin_level0", 'cluster', population_column, grid)

In [93]:
# Create layer with boundaries corresponding to the grid 
gridded_admin_boundaries("admin_level1", 'dissolve_i', population_column, grid)

In [ ]:
r.population.density.py -a -f --overwrite vector=admin_level0@PERMANENT land_cover=landcover@PERMANENT land_use=landuse@PERMANENT tile_size=100 id=Sect_ID population=pop_Total output=Ouaga_lc_lu_100m_weighting_RF plot=/media/tais/My_Book_1/MAUPP/Traitement/Population_modelling_dasymetry/Results/Ouagadougou/RF_feature_importance.png log_file=/media/tais/My_Book_1/MAUPP/Traitement/Population_modelling_dasymetry/Results/Ouagadougou/RF_log n_jobs=8

In [ ]:
v.area.weigh --overwrite vector=admin_level0_100m_gridded@PERMANENT column=pop_Total weight=Ouaga_lc_lu_100m_weighting_RF@PERMANENT output=Ouaga_lc_lu_100m_pop_estimation